## Make an NWB File with Frank Lab data

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

import pynwb

# General dependencies
import os
import numpy as np

# Time
from datetime import datetime
from dateutil import tz

# Helpers for parsing Frank Lab Matlab data
import nspike_helpers as ns 
import query_helpers as qu

# Frank Lab PyNWB extensions and extension-related helpers
import fl_extension as fle
import fl_extension_helpers as flh

# Plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
mdates.rcParams.update({'date.autoformatter.microsecond': '%H:%M:%S.%f'})

### Which data do you want to load?

We will create an NWB file storing data from one experimental day for one animal.

In [2]:
# Path to directory where the Frank Lab CRCNS data is located
data_dir = os.path.expanduser('~/Data/FrankData/KKay/Bon')

day = 4
animal = 'Bon'
    
# Date and time this experiment was actually conducted
dataset_zero_time = datetime(2006, 1, day, 12, 0, 0, tzinfo=tz.gettz('US/Pacific'))

### Initialize the new PyNWB object

The PyNWB object provides an API (i.e. set of easily-used functions) that allow us to store our data in the
appropriate places for eventually saving as a valid NWB file.

In [3]:
nwbf = pynwb.NWBFile(
           session_description='Frank Lab CRCNS data for animal {0}, day {1}'.format(animal, day),
           identifier='{0}{1:04}'.format(animal, day),
           session_start_time=dataset_zero_time,
           file_create_date=datetime.now(tz.tzlocal()),
           lab='Frank Laboratory',
           experimenter='Mattias Karlsson',
           institution='UCSF',
           experiment_description='Recordings from awake behaving rat',
           session_id=data_dir)

print("Here is the top-level structure of our new (still mostly empty) PyNWB object:")
print(nwbf)

Here is the top-level structure of our new (still mostly empty) PyNWB object:

root <class 'pynwb.file.NWBFile'>
Fields:
  acquisition: { }
  analysis: { }
  devices: { }
  electrode_groups: { }
  epoch_tags: {}
  experiment_description: Recordings from awake behaving rat
  experimenter: Mattias Karlsson
  ic_electrodes: { }
  imaging_planes: { }
  institution: UCSF
  lab: Frank Laboratory
  lab_meta_data: { }
  modules: { }
  ogen_sites: { }
  session_id: /Users/ericmiller/Data/FrankData/KKay/Bon
  stimulus: { }
  stimulus_template: { }
  time_intervals: { }



### Initialize PyNWB objects for storing behavioral timeseries data

PyNWB provides several datatypes for specific kinds of behavior data. This allows anyone using PyNWB to know what kinds of data are stored in which places in the PyNWB file. However, all of these are examples of timeseries data (i.e. data with assocaited timestamps). Here, we use the following:
- spatial position (x/y) will be stored in a [Position](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.Position) object
- head direction (angle) will be stored in a [CompassDirection](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.CompassDirection) object
- speed (m/s) will be stored in a more general [BehavioralTimeSeries](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.BehavioralTimeSeries) object

Each of these datatypes are a [MultiContainerInterface](https://pynwb.readthedocs.io/en/latest/pynwb.core.html#pynwb.core.MultiContainerInterface), which just means that we can store multiple instances of the same datatype. For example, a single [BehavioralTimeSeries](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.BehavioralTimeSeries), which we use for storing speed data, can store multiple [TimeSeries](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.TimeSeries) objects, such as for the speed of each epoch. We will use this feature to store the animal's speed for each epoch as a unique [TimeSeries](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.TimeSeries) within the same 
overarching [BehavioralTimeSeries](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.BehavioralTimeSeries).

In [4]:
position = pynwb.behavior.Position(name='Position')
head_dir = pynwb.behavior.CompassDirection(name='Head Direction')
speed = pynwb.behavior.BehavioralTimeSeries(name='Speed')

print("Here are the PyNWB objects where we will store behavior data:")
print(position)
print(head_dir)
print(speed)

Here are the PyNWB objects where we will store behavior data:

Position <class 'pynwb.behavior.Position'>
Fields:
  spatial_series: { }


Head Direction <class 'pynwb.behavior.CompassDirection'>
Fields:
  spatial_series: { }


Speed <class 'pynwb.behavior.BehavioralTimeSeries'>
Fields:
  time_series: { }



### Import animal behavior data
Parse behavioral data from the old Frank Lab Matlab files and store the data in the PyNWB datatypes we initialized above.
We store the behavioral data for each epoch in this day as a separate timeseries within the same overarching PyNWB behavior object. 
For example, we add the animal's 2D spatial position for a given epoch by calling the [create_spatial_series](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html?highlight=create_spatial_series#pynwb.behavior.Position.create_spatial_series) method on the 
Position object, giving it a unique name. Similarly, for speed we use the [create_timeseries](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html?highlight=create_spatial_series#pynwb.behavior.BehavioralTimeSeries.create_timeseries) method on the BehavioralTimeSeries object
to add the animal's speed for this epoch and day.

In [5]:
behavior_data = flh.parse_franklab_behavior_data(data_dir, animal, day)
epoch_time_ivls = []
time_idx, x_idx, y_idx, dir_idx, vel_idx = range(5)  # column ordering of the behavioral data matrix

# Loop over each epoch of this day
for epoch_num, epoch_data in behavior_data.items():

    # Note that we convert timestamps to POSIX time
    timestamps = epoch_data['data'][:,time_idx] + dataset_zero_time.timestamp()

    # Store the times of epoch start and end. We will use these later to build the Epochs table
    epoch_time_ivls.append([timestamps[0], timestamps[-1]])

    # NWB expects meters per pixel
    m_per_pixel = epoch_data['cmperpixel'][0,0]/100   

    # Add this epoch's data to the PyNWB objects: position, head_dir, speed
    position.create_spatial_series(name='Position d%d e%d' % (day, epoch_num), 
                                   timestamps=timestamps,
                                   data=epoch_data['data'][:, (x_idx, y_idx)] * m_per_pixel,
                                   reference_frame='corner of video frame')
    head_dir.create_spatial_series(name='Head Direction d%d e%d'% (day, epoch_num), 
                                   timestamps=timestamps,
                                   data=epoch_data['data'][:, dir_idx],
                                   reference_frame='0=facing top of video frame (?), positive clockwise (?)')
    speed.create_timeseries(name='Speed d%d e%d' % (day, epoch_num),
                            timestamps=timestamps,
                            data=epoch_data['data'][:, vel_idx] * m_per_pixel,
                            unit='m/s',
                            description='smoothed movement speed estimate')
    
print("Here are the same PyNWB objects, now filled with behavior data from each epoch:")
print(position)
print(head_dir)
print(speed)

Here are the same PyNWB objects, now filled with behavior data from each epoch:

Position <class 'pynwb.behavior.Position'>
Fields:
  spatial_series: { Position d4 e1 <class 'pynwb.behavior.SpatialSeries'>,  Position d4 e2 <class 'pynwb.behavior.SpatialSeries'>,  Position d4 e3 <class 'pynwb.behavior.SpatialSeries'>,  Position d4 e4 <class 'pynwb.behavior.SpatialSeries'>,  Position d4 e5 <class 'pynwb.behavior.SpatialSeries'>,  Position d4 e6 <class 'pynwb.behavior.SpatialSeries'>,  Position d4 e7 <class 'pynwb.behavior.SpatialSeries'> }


Head Direction <class 'pynwb.behavior.CompassDirection'>
Fields:
  spatial_series: { Head Direction d4 e1 <class 'pynwb.behavior.SpatialSeries'>,  Head Direction d4 e2 <class 'pynwb.behavior.SpatialSeries'>,  Head Direction d4 e3 <class 'pynwb.behavior.SpatialSeries'>,  Head Direction d4 e4 <class 'pynwb.behavior.SpatialSeries'>,  Head Direction d4 e5 <class 'pynwb.behavior.SpatialSeries'>,  Head Direction d4 e6 <class 'pynwb.behavior.SpatialSeries'>

### Add the PyNWB behavior objects to the main NWBFile object
Now that we have added the behavioral data to separate PyNWB objects (position, head_dir, speed), we need to put these objects into the appropriate place in the main PyNWB.NWBFile object. Specifically, we will add them to a [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule), which is basically just a named bucket where we can store processed data of a particular type.  In our case, we will create a [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule) called "behavior" for storing processed behavior data.

 We use the [add_data_interface](https://pynwb.readthedocs.io/en/latest/pynwb.core.html#pynwb.core.NWBDataInterface) method to add each of our behavior objects to the [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule). The term "data interface" refers to an [NWBDataInterface](https://pynwb.readthedocs.io/en/latest/pynwb.core.html#pynwb.core.NWBDataInterface), which you might come across in reading the documentation. This terminology can be quite confusing, so we will reiterate what the terms mean:
- a [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule) is a named bucket where we put processed data of a particular type (e.g. behavior data)
- an [NWBDataInterface](https://pynwb.readthedocs.io/en/latest/pynwb.core.html#pynwb.core.NWBDataInterface) is any datatype that we want to store in a [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule). All of the datatypes that we used to store our behavior data are examples of this.

One other thing to note here is the distinction between the PyNWB NWBFile object and the NWB file that we are creating. We have not yet saved anything to an NWB file on disk. We will do that at the end, after we have added everything to the PyNWB NWBFile object in the appropriate locations.

In [6]:
# create a ProcessingModule for behavior data
behav_mod = nwbf.create_processing_module(name='Behavior', 
                                          description='Behavioral data')

# Add the position, head direction and speed data to the ProcessingModule
behav_mod.add_data_interface(position)
behav_mod.add_data_interface(head_dir)
behav_mod.add_data_interface(speed)

print("Here is the ProcessingModule (i.e. bucket in the NWB file) where we will store all of the behavior data:")
print(behav_mod)

Here is the ProcessingModule (i.e. bucket in the NWB file) where we will store all of the behavior data:

Behavior <class 'pynwb.base.ProcessingModule'>
Fields:
  data_interfaces: { Head Direction <class 'pynwb.behavior.CompassDirection'>,  Position <class 'pynwb.behavior.Position'>,  Speed <class 'pynwb.behavior.BehavioralTimeSeries'> }
  description: Behavioral data



### Represent behavioral tasks using Frank Lab NWB extension
We represent each behavioral task as a Frank Lab Task (franklab.extensions.yaml). This object simply contains a name and a detailed description of the task. For the dataset here, we only have two tasks: W-Alternation and Sleep.

We then store Task objects in the "Behavior" [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule), just like we did with the other behavioral data above.

In [7]:
task_name = 'Sleep'
description = 'The animal sleeps or wanders freely around a small, empty box.'
behav_mod.add_data_interface(fle.Task(name=task_name, description=description))

task_name = 'W-Alternation'
task_description = 'The animal runs in an alternating W pattern between three neighboring arms of a maze.'
behav_mod.add_data_interface(fle.Task(name=task_name, description=task_description))

print("Note that we've added fl_extension.Task objects to our ProcessingModule:")
print(behav_mod)

Note that we've added fl_extension.Task objects to our ProcessingModule:

Behavior <class 'pynwb.base.ProcessingModule'>
Fields:
  data_interfaces: { Head Direction <class 'pynwb.behavior.CompassDirection'>,  Position <class 'pynwb.behavior.Position'>,  Sleep <class 'fl_extension.Task'>,  Speed <class 'pynwb.behavior.BehavioralTimeSeries'>,  W-Alternation <class 'fl_extension.Task'> }
  description: Behavioral data



### Represent behavioral apparatuses using Frank Lab NWB extension
Accurately representing the behavioral apparatuses (i.e. tracks, mazes, open fields, sleep boxes) is essential for interpreting the data. For the dataset here, we have three apparatuses: Sleep Box, W-track A, and W-track B. We represent each behavioral apparatus as a Frank Lab Apparatus (franklab.extensions.yaml), which uses a graph representation (i.e. nodes and edges) to represent the topology of a track.

A Frank Lab Apparatus consists of a set of Nodes and Edges (i.e. it is a graph) describing the topology of the apparatus. Each Node represents an important component of the apparatus. For example:
- PointNode represents 0D point with an x/y position (e.g. reward well, novel object)
- SegmentNode represents a 1D line segment (e.g. linearized maze arm)
- PolygonNode represents a 2D area (e.g. open field / non-linearizable area)

Each Node object has a set of coordinates that describes its spatial geometry (e.g. the vertices for a PolygonNode). Nodes sharing at least one coordinate can be represented as spatially connected by adding an Edge. For example, we can represent a reward well (PointNode) at the end of a linearized W-track arm (SegmentNode) by adding an Edge connecting those nodes. 

In [8]:
# ---------
# Apparatus 1: Sleep Box
#   The Sleep Box consists of a single PolygonNode representing the polygon area of the box.
#   The vertices of this polygon are defined in the "coords" property of the PolygonNode.
#   There are no edges since there is only one node.
# ---------
sleepbox_nodes = flh.get_sleepbox_nodes() 
sleepbox_edges = []  # Sleep box consists of only one Node, so it has no Edges by definition.
sleep_box_apparatus = fle.Apparatus(name='Sleep Box', nodes=sleepbox_nodes, edges=sleepbox_edges)


# ---------
# Apparatus 2: W-track A
#   This apparatus consists of several SegmentNodes representing the linearized segements of the W-track
#   and three PointNodes representing the reward wells at the end of the three "arms" of the track.
#   There are Edges connecting the SegmentNodes and PointNodes according to the topology of the
#   track (i.e. edges connecting track components that are, in fact, connected spatially).
# ---------
wtrack_A_nodes = flh.get_wtrack_A_nodes()
wtrack_A_edges = flh.find_edges(wtrack_A_nodes) # finds Nodes with 1 or more shared x/y coords
wtrack_A_apparatus = fle.Apparatus(name='W-track A', nodes=wtrack_A_nodes, edges=wtrack_A_edges)


# ---------
# Apparatus 3: W-track B
#   The second W-track used in this experiment has the same basic structure as W-track A, but
#   its geometry is distinct and thus should be represented as a separate Apparatus object.
# ---------
wtrack_B_nodes = flh.get_wtrack_B_nodes()  
wtrack_B_edges = flh.find_edges(wtrack_B_nodes)
wtrack_B_apparatus = fle.Apparatus(name='W-track B', nodes=wtrack_B_nodes, edges=wtrack_B_edges)


print("Here are the three Apparatus objects. Notice that they consist of 'edges' and 'nodes'.")
print(sleep_box_apparatus)
print(wtrack_A_apparatus)
print(wtrack_B_apparatus)

Here are the three Apparatus objects. Notice that they consist of 'edges' and 'nodes'.

Sleep Box <class 'fl_extension.Apparatus'>
Fields:
  edges: { }
  nodes: { Sleep box polygon <class 'fl_extension.PolygonNode'> }


W-track A <class 'fl_extension.Apparatus'>
Fields:
  edges: { segment1<->segment2 <class 'fl_extension.Edge'>,  segment1<->segment4 <class 'fl_extension.Edge'>,  segment1<->well1 <class 'fl_extension.Edge'>,  segment2<->segment1 <class 'fl_extension.Edge'>,  segment2<->segment3 <class 'fl_extension.Edge'>,  segment2<->segment4 <class 'fl_extension.Edge'>,  segment3<->segment2 <class 'fl_extension.Edge'>,  segment3<->well2 <class 'fl_extension.Edge'>,  segment4<->segment1 <class 'fl_extension.Edge'>,  segment4<->segment2 <class 'fl_extension.Edge'>,  segment4<->segment5 <class 'fl_extension.Edge'>,  segment5<->segment4 <class 'fl_extension.Edge'>,  segment5<->well3 <class 'fl_extension.Edge'>,  well1<->segment1 <class 'fl_extension.Edge'>,  well2<->segment3 <class 'fl_ex

### Store the Apparatuses in the NWBFile object

After building the Apparatus objects, we store them in the "Behavior" [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule), just like we did with the other behavioral data above.

In [9]:
# ---------
# Add all three Apparatuses to the "Behavior" ProcessingModule
# ---------
behav_mod.add_data_interface(sleep_box_apparatus)
behav_mod.add_data_interface(wtrack_A_apparatus)
behav_mod.add_data_interface(wtrack_B_apparatus)

print("Note that our fl_extension.Apparatus objects are in the ProcessingModule:")
print(behav_mod)

Note that our fl_extension.Apparatus objects are in the ProcessingModule:

Behavior <class 'pynwb.base.ProcessingModule'>
Fields:
  data_interfaces: { Head Direction <class 'pynwb.behavior.CompassDirection'>,  Position <class 'pynwb.behavior.Position'>,  Sleep <class 'fl_extension.Task'>,  Sleep Box <class 'fl_extension.Apparatus'>,  Speed <class 'pynwb.behavior.BehavioralTimeSeries'>,  W-Alternation <class 'fl_extension.Task'>,  W-track A <class 'fl_extension.Apparatus'>,  W-track B <class 'fl_extension.Apparatus'> }
  description: Behavioral data



### Store epoch metadata in the NWBFile object
We store metadata in the top-level [NWBFile.epochs](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.epochs) table. By default, there are required columns for 'start_time', 'stop_time', and 'tags'. We add additional metadata columns for the epoch's Task, Apparatus, etc. using the [NWBFile.add_epoch_column()](https://pynwb.readthedocs.io/en/latest/pynwb.file.html#pynwb.file.NWBFile.add_epoch_column) method. After we have all of the metadata columns set up, we add each epoch as a new row of the table using the [NWBFile.add_epoch()](https://pynwb.readthedocs.io/en/latest/pynwb.file.html#pynwb.file.NWBFile.add_epoch) method.

<i>Take a look under the PyNWB hood</i>:</br>
Each epoch occurs in a discrete time interval defined by its start and stop times. As such, the [NWBFile.epochs](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.epochs) table is an instance of [TimeIntervals](https://pynwb.readthedocs.io/en/latest/pynwb.epoch.html?highlight=TimeIntervals#pynwb.epoch.TimeIntervals), which is itself an instance of [DynamicTable](https://pynwb.readthedocs.io/en/latest/pynwb.core.html#pynwb.core.DynamicTable). Later, we will store electrodes and clustered units in two other DynamicTables. One of the advantages of using DynamicTables is it allows for adding arbitrary columns.

In [10]:
# ---------
# Load epochs metadata from the Frank Lab Matlab files
# ---------
all_epochs_metadata = flh.parse_franklab_task_data(data_dir, animal, day)

# ---------
# Add metadata columns to the NWBFile.epochs table
# By default, ithas columns for 'start_time', 'stop_time', and 'tags'.
# ---------
nwbf.add_epoch_column(name='exposure', description='number of exposures to this environment')
nwbf.add_epoch_column(name='task', description='behavioral task for this epoch')
nwbf.add_epoch_column(name='apparatus', description='behavioral apparatus for this epoch')

# ---------
# Iteratively add each epoch to the NWBFile.epochs table
# ---------
for epoch_num, epoch_metadata in all_epochs_metadata.items():
    
    # start and stop times were inferred from the behavior data earlier
    epoch_start_time, epoch_stop_time = epoch_time_ivls[epoch_num-1]  
    
    # meter per pixel ratio is also in the behavior data
    m_per_pixel = behavior_data[epoch_num]['cmperpixel'][0,0]/100  
    
    # Frank Lab Task (from the "Behavior" ProcessingModule)
    epoch_task = flh.get_franklab_task(epoch_metadata, behav_mod)
    
    # Frank Lab Apparatus (from the "Behavior" ProcessingModule)
    epoch_apparatus = flh.get_franklab_apparatus(epoch_metadata, behav_mod)
    
    epoch_exposure_num = flh.get_exposure_num(epoch_metadata)
    
    # Required column 'tags'. We do not presently use this.
    epoch_tags = ''
    
    # Add this epoch to the NWBFile.epochs table
    # Note that task and apparatus are soft links to the "Behavior" ProcessingModule, 
    # so they will not be unnecessarily duplicated within the NWBFile
    nwbf.add_epoch(start_time=epoch_start_time,
                   stop_time=epoch_stop_time,
                   exposure=epoch_exposure_num,
                   task=epoch_task,
                   apparatus=epoch_apparatus,
                   tags=epoch_tags)


print("Here is an example epoch from the table.\nNote that the 'task' and 'apparatus' columns point to our Frank Lab extension objects.\n")
print(nwbf.epochs.to_dataframe().iloc[0, :])

# print("Epoch number: {}".format(nwbf.epochs[1][0] + 1))
# print("Start, stop times: {0}, {1}".format(nwbf.epochs[1][1], nwbf.epochs[1][2]))
# print("Exposure number: {}".format(nwbf.epochs[1][3]))
# print("Task name: {}".format(nwbf.epochs[1][4].name))
# print("Apparatus name: {}".format(nwbf.epochs[1][5].name))

Here is an example epoch from the table. Note that the 'task' and 'apparatus' columns point to our Frank Lab extension objects.

start_time                                          1.13641e+09
stop_time                                           1.13641e+09
exposure                                                     NA
task          \nSleep <class 'fl_extension.Task'>\nFields:\n...
apparatus     \nSleep Box <class 'fl_extension.Apparatus'>\n...
tags                                                         []
Name: 0, dtype: object


### Process tetrodes metadata

1. Each tetrode gets an [ElectrodeGroup](https://pynwb.readthedocs.io/en/latest/pynwb.ecephys.html#pynwb.ecephys.ElectrodeGroup), where we store metadata about the tetrode such a unique name and the region of the brain it's in. 

2. Each individual recording channel (i.e. each of the 4 tetrode channels) gets its own row in the top-level [NWBFile.electrodes](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.electrodes) table. Here we store metadata about the electrode such as its location/depth in the brain, and the tetrode that it is part of.  We use the [NWBFile.add_electrode()](https://pynwb.readthedocs.io/en/latest/pynwb.file.html#pynwb.file.NWBFile.add_electrode) method to add each channel. As with the NWBFile.epochs table, discussed above, this is a DynamicTable.

3. Each tetrode gets an "Electrode Table Region", an instance of [DynamicTableRegion](https://pynwb.readthedocs.io/en/latest/pynwb.core.html?highlight=DynamicTableRegion#pynwb.core.DynamicTableRegion). This is just a slice into the [NWBFile.electrodes](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.electrodes) table selecting the channels that go with the tetrode. We use the [create_electrode_table_region()]() method to add each tetrode's Electrode Table Region.

4. Since LFP is often taken from a subset of a tetrode's channels, we create an Electrode Table Region for each tetrode's LFP channles. For example, if LFP was taken from just the first channel of a tetrode, we would create an Electrode Table Region just pointing to that channel of the [NWBFile.electrodes](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.electrodes) table.

In [12]:
# Parse tetrodes metadata from the old Frank Lab Matlab files
tetrode_metadata = flh.parse_franklab_tetrodes(data_dir, animal, day)

# Represent our acquisition system with a 'Device' object
recording_device = nwbf.create_device(name='NSpike acquisition system')

# Four channels per tetrode by definition 
num_chan_per_tetrode = 4     

# Initialize dictionaries to store the metadata
tet_electrode_group = {}  # group for each tetrode
tet_electrode_table_region = {}  # region for each tetrode (all channels)
lfp_electrode_table_region = {}  # region for each tetrode's LFP channels

chan_num = 0   # Incrementing channel number
for tet_num, tet in tetrode_metadata.items():
    
    # Define some metadata parameters
    tetrode_name = "%02d-%02d" % (day, tet_num) 
    impedance = np.nan
    filtering = 'unknown - likely 600Hz-6KHz'
    location = flh.get_franklab_tet_location(tet)  # area/subarea in the brain
    coord = flh.get_franklab_tet_coord(tet)  # x/y/z coordinate in the brain
    description = "tetrode {tet_num} located in {location} on day {day}".format(
        tet_num=tet_num, location=location, day=day)
    
    # 1. Represent the tetrode in NWB as an ElectrodeGroup
    tet_electrode_group[tet_num] = nwbf.create_electrode_group(name=tetrode_name,
                                                               description=description,
                                                               location=location,
                                                               device=recording_device)
    
    # 2. Represent each channels of the tetrode as a row in the NWBFile.electrodes table.
    for i in range(num_chan_per_tetrode):
            nwbf.add_electrode(x=coord[0],
                               y=coord[1],
                               z=coord[2],
                               imp=impedance,
                               location=location,
                               filtering=filtering,
                               group=tet_electrode_group[tet_num],  # tetrode this electrode belongs to
                               group_name=tet_electrode_group[tet_num].name,
                               id=chan_num)
            chan_num = chan_num + 1  # total number of channels processed so far across all tets
            
    # 3. Create an Electrode Table Region (slice into the electrodes table) for each tetrode
    table_region_description = 'tetrode %d all channels' % tet_num
    table_region_name = '%d' % tet_num
    table_region_rows = list(range(chan_num - num_chan_per_tetrode, chan_num)) # rows of NWBFile.electrodes table
    tet_electrode_table_region[tet_num] = nwbf.create_electrode_table_region(
        region=table_region_rows,
        description=table_region_description,
        name=table_region_name)

    # 4. Create an ElectrodeTableRegion for each tetrode's LFP recordings
    lfp_channels = [chan_num - num_chan_per_tetrode] # Assume that LFP is taken from the first channel
    table_region_description = 'tetrode %d LFP channels' % tet_num
    lfp_electrode_table_region[tet_num] = nwbf.create_electrode_table_region(
        region=lfp_channels,
        description=table_region_description,
        name=table_region_name)

print("Here is an example ElectrodeGroup for a tetrode: ")
print(tet_electrode_group[1])
print('\n')
print("Here is the ElectrodeTableRegion for that tetrode:")
print(tet_electrode_table_region[1])
print('\n')
print("Here is an example channel from the NWBFile.electrodes table:")
print(nwbf.electrodes.to_dataframe().iloc[0, :])

Here is an example ElectrodeGroup for a tetrode: 

04-01 <class 'pynwb.ecephys.ElectrodeGroup'>
Fields:
  description: tetrode 1 located in CA3 on day 4
  device: NSpike acquisition system <class 'pynwb.device.Device'>
  location: CA3



Here is the ElectrodeTableRegion for that tetrode:

1 <class 'pynwb.core.DynamicTableRegion'>
Fields:
  description: tetrode 1 all channels
  table: electrodes <class 'pynwb.core.DynamicTable'>



Here is an example channel from the NWBFile.electrodes table:
x                                                           NaN
y                                                           NaN
z                                                       3.94229
imp                                                         NaN
location                                                    CA3
filtering                           unknown - likely 600Hz-6KHz
group         \n04-01 <class 'pynwb.ecephys.ElectrodeGroup'>...
group_name                                             

In [ ]:
# Path to the EEG data sub-directory
eeg_subdir = "EEG"
eeg_path = os.path.join(data_dir, eeg_subdir)
if not os.path.exists(eeg_path):
    raise new RuntimeException('Error: eeg_path %s does not exist' % eeg_path)
    
day_str = '%02d' % day 
nwb_filename = anim + day_str + '.nwb'

prefix = anim.lower()

eeg_subdir = "EEG"
eeg_path = os.path.join(data_dir, eeg_subdir)

# Specific file names
epochs_file = "times.mat"
tetinfo_file = "tetinfo.mat"